In [1]:
import pandas as pd
import numpy as np

In [2]:
# Create the team dataframe
team_data = {
    'id': ['A', 'B', 'C', 'D'],
    'teamname': ['Team A', 'Team B', 'Team C', 'Team D'],
    'coach': ['Coach A', 'Coach B', 'Coach C', 'Coach D']
}
team_df = pd.DataFrame(team_data)

# Create the goal dataframe
goal_data = {
    'matchid': [101, 101, 101, 102],
    'teamid': ['A', 'A', 'B', 'D'],
    'player': ['A1', 'A9', 'B7', 'D10'],
    'goal_time': [17, 58, 89, 63]
}
goal_df = pd.DataFrame(goal_data)

# Create the game dataframe
game_data = {
    'id': [101, 102, 103, 104],
    'mdate': pd.to_datetime(['2019-01-04', '2019-01-04', '2019-01-10', '2019-01-13']),
    'stadium': ['stadium 1', 'stadium 3', 'stadium 1', 'stadium 2'],
    'team1': ['A', 'D', 'A', 'B'],
    'team2': ['B', 'E', 'C', 'E']
}
game_df = pd.DataFrame(game_data)

print(team_df)
print(goal_df)
print(game_df)

  id teamname    coach
0  A   Team A  Coach A
1  B   Team B  Coach B
2  C   Team C  Coach C
3  D   Team D  Coach D
   matchid teamid player  goal_time
0      101      A     A1         17
1      101      A     A9         58
2      101      B     B7         89
3      102      D    D10         63
    id      mdate    stadium team1 team2
0  101 2019-01-04  stadium 1     A     B
1  102 2019-01-04  stadium 3     D     E
2  103 2019-01-10  stadium 1     A     C
3  104 2019-01-13  stadium 2     B     E


Merging game and goal tables

Using left_on & right_on since join columns have different names 

In [3]:
merged_game_goal = pd.merge(game_df, goal_df, how='left', left_on='id' ,right_on='matchid')
merged_game_goal

,id,mdate,stadium,team1,team2,matchid,teamid,player,goal_time
0,101,2019-01-04,stadium 1,A,B,101.0,A,A1,17.0
1,101,2019-01-04,stadium 1,A,B,101.0,A,A9,58.0
2,101,2019-01-04,stadium 1,A,B,101.0,B,B7,89.0
3,102,2019-01-04,stadium 3,D,E,102.0,D,D10,63.0
4,103,2019-01-10,stadium 1,A,C,NaN,NaN,NaN,NaN
5,104,2019-01-13,stadium 2,B,E,NaN,NaN,NaN,NaN


In [9]:
# Another way
# merged_game_goal['t1_flag'] = merged_game_goal.apply(lambda x: 't' if x['team1'] == x['teamid'] else 'f', axis=1)

merged_game_goal['t1_flag'] = (merged_game_goal['team1'] == merged_game_goal['teamid']).astype(int)
merged_game_goal['t2_flag'] = (merged_game_goal['team2'] == merged_game_goal['teamid']).astype(int)


In [20]:
# res = merged_game_goal.groupby(['id','mdate','team1','team2'], as_index= False).agg({'t1_flag' : 'sum', 't2_flag' : 'sum'}).drop(columns='id').rename(columns={'t1_flag': 't1_score', 't2_flag': 't2_score'})


# Group by id, mdate, team1, team2 and then perform aggregations
res = (merged_game_goal
       .groupby(['id','mdate', 'team1', 'team2'], as_index=False)
       .agg(t1_score=('t1_flag', 'sum'), t2_score=('t2_flag', 'sum'))  # Renaming columns along with aggregation
)

# Re-ordering as per our needs
print(res[['mdate','team1','t1_score','team2','t2_score']])


       mdate team1  t1_score team2  t2_score
0 2019-01-04     A         2     B         1
1 2019-01-04     D         1     E         0
2 2019-01-10     A         0     C         0
3 2019-01-13     B         0     E         0
